In [185]:
import numpy as np
import scipy.stats as stats
import sympy as sp 

def y(ti, dti, Na, Nb,l_a = None, l_b = None):
    '''
    Calcula o valor de uma função y em função de Na, Nb, l_a, l_b, ti e dti.
    ''' 
    return Na*np.exp(-l_a*ti)*(1 - np.exp(-l_a*dti))+Nb*np.exp(-l_b*ti)*(1 - np.exp(-l_b*dti))

def derivadas_y(ti, dti, Na, Nb,l_a = None, l_b = None): 
    '''
    Calcula as derivadas parciais de uma função y em relação às variáveis Na, Nb, l_a e l_b.
    '''
    y = Na*sp.exp(-l_a*ti)*(1 - sp.exp(-l_a*dti))+Nb*sp.exp(-l_b*ti)*(1 - sp.exp(-l_b*dti))
    dNa = y.diff(Na)
    dNb = y.diff(Nb)
    dla = y.diff(l_a)
    dlb = y.diff(l_b)
    return dNa, dNb, dla, dlb

def incerteza(derivadas, sigma_Na, sigma_Nb, sigma_la, sigma_lb, sigma_Na_Nb): 
    '''
    Calcula a incerteza de uma função y em relação às variáveis Na, Nb, l_a e l_b.
    '''
    dNa, dNb, dla, dlb = derivadas
    sy = (dNa**2)*sigma_Na**2 + (dNb**2)*sigma_Nb**2 + (dla**2)*sigma_la**2 + (dlb**2)*sigma_lb**2 + 2*dNa*dNb*sigma_Na_Nb
    return sy


def teste_chi2_direita(chi_quadrado, graus_de_liberdade, nivel_significancia_sup):
    '''
    chi2: chi2 do ajuste
    graus_de_liberdade: graus de liberdade do ajuste
    nivel_significancia_sup: nível de significância do teste
    '''

    # valores críticos
    valor_critico_sup = stats.chi2.ppf(1 - nivel_significancia_sup, graus_de_liberdade)


    qui_quadrado_reduzido_sup = valor_critico_sup / graus_de_liberdade
    print(f"Limite superior do X²_red: {qui_quadrado_reduzido_sup:.4f}")
    print(f"O X²_red obtido: {chi_quadrado/graus_de_liberdade}")

    if chi_quadrado/graus_de_liberdade >= qui_quadrado_reduzido_sup:
        print('Falhou no teste')
    else:
        print('Passou no teste')

In [186]:
ti, dti, Na, Nb, l_a, l_b = sp.symbols('ti dti Na Nb la lb')
derivadas = derivadas_y(ti, dti, Na, Nb, l_a, l_b) # derivadas parciais de y em relação a Na, Nb, l_a e l_b

sigma_Na, sigma_Nb, sigma_la, sigma_lb, sigma_Na_Nb = sp.symbols('sigma_Na sigma_Nb sigma_la sigma_lb sigma_Na_Nb')

incertezas = incerteza(derivadas_y(ti, dti, Na, Nb, l_a, l_b), sigma_Na, sigma_Nb, sigma_la, sigma_lb, sigma_Na_Nb) # incerteza de y 

incertezas_func = sp.lambdify((ti, dti, Na, Nb, l_a, l_b, sigma_Na, sigma_Nb, sigma_la, sigma_lb, sigma_Na_Nb), incertezas, 'numpy') # incerteza de y em um formato que aceita numpy



In [187]:
#abre o arquivo
import pandas as pd
import os
data = pd.read_csv(r'C:\Users\Gustavo\Documents\GitHub\Estatastica-com-Python\dadosAtividadePratica_4.txt', delimiter='\t')
print(data)
ti = np.array(data['t(h)'])
dti = np.array(data['dt(h)'])
y_data = np.array(data['n'])
s_y = np.array(data['sigma'])

   t(h)  dt(h)      n  sigma
0   0.5    1.5  16183    133
1   2.2    1.5   5453     84
2   3.9    1.5   2646     65
3   5.5    3.0   2900     76
4   8.7    4.0   1688     76


In [188]:
#faz o ajuste 
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

la = 0.203 #lambdas 
lb = 0.92
D = 0.0001 #delta

la1 = la + D #lambda '
lb2 = lb + D #lambda ''

V_l = (10**(-4))*np.array([[0.81, 0], [0, 3.24]]) #matriz de covariância de l_a e l_b

ti_dti_data = np.vstack((ti, dti)) #transforma ti e dti em um array de arrays

#MMQ e Chi2
N, V_y = curve_fit(lambda td, Na, Nb: y(td[0], td[1], Na, Nb, l_a = la, l_b = lb), ti_dti_data, y_data, absolute_sigma = True, sigma = s_y) 

print('N: ', N)
print('V_y: ', V_y, '\n')

chi_quadrado_ant = np.sum(((y_data - y(ti, dti, N[0], N[1], l_a = la, l_b = lb))**2)/(s_y**2))

teste_chi2_direita(chi_quadrado_ant, len(ti) - 2, 0.01)

#MMQ para lambda ' e lambda ''
N1, V_y1 = curve_fit(lambda td, Na, Nb: y(td[0], td[1], Na, Nb, l_a = la1, l_b = lb), ti_dti_data, y_data, absolute_sigma = True, sigma = s_y)


N2, V_y2 = curve_fit(lambda td, Na, Nb: y(td[0], td[1], Na, Nb, l_a = la, l_b = lb2), ti_dti_data, y_data, absolute_sigma = True, sigma = s_y)


# parâmetros em cada etapa 
N_a, N_b = N
N_1a, N_1b = N1
N_2a, N_2b = N2 

#definição da matriz F
F11 = (N_1a - N_a)/D
F12 = (N_2a - N_a)/D

F21 = (N_1b - N_b)/D
F22 = (N_2b - N_b)/D

F = np.array([[F11, F12],[F21, F22]])

#Calculo de V_final
V_final_D1 = V_y + F @ V_l @ F.T



print('\n', 'V_final: ', V_final_D1)

N:  [17969.37515015 25172.24950633]
V_y:  [[103617.53419688 -69237.4869057 ]
 [-69237.4869057  117122.64295431]] 

Limite superior do X²_red: 3.7816
O X²_red obtido: 0.4707473337867436
Passou no teste

 V_final:  [[ 212005.20295158 -195837.62255466]
 [-195837.62255466  349409.55978059]]


In [189]:
# Calculo da função no ponto mais sua incerteza
y_df = []
sy_df = []


for i, element in enumerate(ti):

    y_df.append(y(element, dti[i], N[0], N[1], l_a = la, l_b = lb))
    incerteza_val = np.sqrt(incertezas_func(element, dti[i], N[0], N[1], la, lb, np.sqrt(V_y[0][0]), np.sqrt(V_y[1][1]),np.sqrt(V_l[0][0]), np.sqrt(V_l[1][1]), V_y[0][1]))
    
    sy_df.append(float(incerteza_val))

In [190]:
#Acrescenta na tabela
data['N_ajuste'] = y_df
data['sigma_N_ajuste'] = sy_df
data['N_ajuste']  = data['N_ajuste'].astype(int)
data['sigma_N_ajuste'] = data['sigma_N_ajuste'].astype(int)
print(data)

   t(h)  dt(h)      n  sigma  N_ajuste  sigma_N_ajuste
0   0.5    1.5  16183    133     16154             191
1   2.2    1.5   5453     84      5507             102
2   3.9    1.5   2646     65      2658              47
3   5.5    3.0   2900     76      2833              67
4   8.7    4.0   1688     76      1716              90


In [191]:
#converte tabela em formato latex
tabela = data.to_latex(index=False) 
print(tabela)

\begin{tabular}{rrrrrr}
\toprule
t(h) & dt(h) & n & sigma & N_ajuste & sigma_N_ajuste \\
\midrule
0.500000 & 1.500000 & 16183 & 133 & 16154 & 191 \\
2.200000 & 1.500000 & 5453 & 84 & 5507 & 102 \\
3.900000 & 1.500000 & 2646 & 65 & 2658 & 47 \\
5.500000 & 3.000000 & 2900 & 76 & 2833 & 67 \\
8.700000 & 4.000000 & 1688 & 76 & 1716 & 90 \\
\bottomrule
\end{tabular}



In [192]:
#teste para \Delta diferente 

from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

la = 0.203
lb = 0.92
D = 0.001

la1 = la + D
lb2 = lb + D

V_l = (10**(-4))*np.array([[0.81, 0], [0, 3.24]])

ti_dti_data = np.vstack((ti, dti))

N, V_y = curve_fit(lambda td, Na, Nb: y(td[0], td[1], Na, Nb, l_a = la, l_b = lb), ti_dti_data, y_data, absolute_sigma = True, sigma = s_y)

chi_quadrado_ant = np.sum(((y_data - y(ti, dti, N[0], N[1], l_a = la, l_b = lb))**2)/(s_y**2))

teste_chi2_direita(chi_quadrado_ant, len(ti) - 2, 0.05)

N1, V_y1 = curve_fit(lambda td, Na, Nb: y(td[0], td[1], Na, Nb, l_a = la1, l_b = lb), ti_dti_data, y_data, absolute_sigma = True, sigma = s_y)


N2, V_y2 = curve_fit(lambda td, Na, Nb: y(td[0], td[1], Na, Nb, l_a = la, l_b = lb2), ti_dti_data, y_data, absolute_sigma = True, sigma = s_y)


N_a, N_b = N
N_1a, N_1b = N1
N_2a, N_2b = N2 

F11 = (N_1a - N_a)/D
F12 = (N_2a - N_a)/D

F21 = (N_1b - N_b)/D
F22 = (N_2b - N_b)/D

F = np.array([[F11, F12],[F21, F22]])

V_final_D2 = V_y + F @ V_l @ F.T
print(V_final_D2)


Limite superior do X²_red: 2.6049
O X²_red obtido: 0.4707473337867436
Passou no teste
[[ 212288.52931858 -196268.7322835 ]
 [-196268.7322835   349707.4125486 ]]


In [193]:
#verificação da diferença do delta no resultado final em porcentagem  
print(100*(V_final_D2 - V_final_D1)/V_final_D1)

[[0.13364123 0.22013632]
 [0.22013632 0.0852446 ]]
